<a href="https://colab.research.google.com/github/therealthaibinh/reference-creator/blob/master/FactCheck_references.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run this cell once at the beginning when you open up this notebook:

In [ ]:
from IPython.core.display import display, HTML

from datetime import datetime

try:
    import requests
except:
    !pip install requests
    import requests
    
try:
    from bs4 import BeautifulSoup
except:
    !pip install beautifulsoup4
    from bs4 import BeautifulSoup

# **Change the URL here, then run the cell:**
- Remember that the URL needs to be inside double-quotes


In [ ]:
lsturl_sub = ["https://www.factcheck.org/2020/08/republican-convention-night-3/", 
              "https://www.factcheck.org/2020/08/final-night-of-the-republican-convention/"]

# Run this cell whenever you change the URL:

In [ ]:
for url_sub in lsturl_sub:
  page_sub = BeautifulSoup(requests.get(url_sub).content, 'html.parser')

  # Author
  lstAuthors = []
  for strAuthor in page_sub.findAll(attrs={'class' : 'author url fn'}):
      lstAuthors.append(strAuthor.contents[0])
  if len(lstAuthors)==1:
      strFirstAuthor = lstAuthors[0]
      strAuthor = strFirstAuthor[strFirstAuthor.find(' ')+1:] + ', ' + strFirstAuthor[:strFirstAuthor.find(' ')]
  elif len(lstAuthors)==2:
      strFirstAuthor = lstAuthors[0]
      strSecondAuthor = lstAuthors[1]
      strAuthor = strFirstAuthor[strFirstAuthor.find(' ')+1:] + ', ' + strFirstAuthor[:strFirstAuthor.find(' ')] + ' and ' +\
                  strSecondAuthor[strSecondAuthor.find(' ')+1:] + ', ' + strSecondAuthor[:strSecondAuthor.find(' ')]
  else:
      strFirstAuthor = lstAuthors[0]
      strAuthor = strFirstAuthor[strFirstAuthor.find(' ')+1:] + ', ' + strFirstAuthor[:strFirstAuthor.find(' ')] + ' et. al'
      
  # Title
  strTitle = page_sub.find(attrs={'class' : 'entry-title'}).contents[0]
  # strTitle_tagged = 

  # Date
  strDate = page_sub.find(attrs={'class' : 'entry-date published updated'}).contents[0]
  strDate = datetime.strptime(strDate, '%B %d, %Y').strftime("%-d %b %Y")

  # Put it all together
  strFCReference = '{AUTHOR}. "{TITLE}." FactCheck.org. {DATE}.'

  # # Plain text with no hyperlinks or nothin'
  # print(strFCReference.format(AUTHOR = strAuthor,
  #                             TITLE = strTitle,
  #                             DATE = strDate))
  # print()

  # Reference with embedded hyperlink
  strReference_tagged = strFCReference.format(AUTHOR = strAuthor,
                                              TITLE = "<a href='{href}'>"+strTitle+"</a>",
                                              DATE = strDate)
  display(HTML(strReference_tagged.format(href=url_sub)))
  # print()

  # # Reference with HTML hyperlink tags
  # print(strReference_tagged.format(href=url_sub))
